In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

In [2]:
df = pd.read_csv('/Users/yelderiny/Projects/Dissertation/Data/processed-data3.csv')
df.head()

,pull_requests,size,contributors,age,contributor_xp1,contributor_xp2,contributor_xp3,language_C++,language_Go,language_Java,language_JavaScript,language_Python,language_Swift,language_TypeScript,pr_points1,pr_points2,pr_points3
0,-0.299689,-0.126432,-0.605191,-1.390139,-0.721174,-0.710998,-0.684408,0,0,1,0,0,0,0,2.271,2.389,2.377
1,-0.276385,-0.234160,-0.649719,-1.466477,-0.322326,-0.345167,-0.597730,0,0,1,0,0,0,0,3.390,3.318,2.749
2,-0.254689,-0.124061,-0.320208,-0.155424,0.060942,0.036422,-0.041027,0,0,1,0,0,0,0,4.006,3.668,3.544
3,-0.323796,-0.239047,-0.756588,-0.782989,-0.652968,-0.668599,-0.811847,0,0,1,0,0,0,0,3.281,2.569,3.257
4,-0.311742,-0.194809,-0.551756,-0.751034,-0.753026,-0.761651,-0.831452,0,0,1,0,0,0,0,4.867,4.833,5.186


In [3]:
num_bins = int(np.ceil(np.log2(len(df['pr_points1'])) + 1))
print(num_bins)

12


In [4]:
# Sturges' Rule 
df['pr_points_bucket'], bins = pd.cut(df['pr_points1'], bins=num_bins, retbins=True, labels=False)
print(bins)

[ 0.926841    7.09658333 13.19316667 19.28975    25.38633333 31.48291667
 37.5795     43.67608333 49.77266667 55.86925    61.96583333 68.06241667
 74.159     ]


In [5]:
features = df.drop(columns=['pr_points1', 'pr_points2', 'pr_points3', 'contributor_xp1', 'contributor_xp3', 'pr_points_bucket'], axis=1)
target = df['pr_points_bucket']

In [6]:
results = {}

In [7]:
def summarize_classification(y_test, y_pred):
    acc = accuracy_score(y_test, y_pred, normalize=True)  # As a fraction
    num_acc = accuracy_score(y_test, y_pred, normalize=False)  # As a count
    prec = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
    
    return {'accuracy': acc, 'precision': prec, 'recall': recall, 'f1': f1, 'accuracy_count': num_acc}

In [8]:
def build_model(classifier_fn, name_of_y_col, name_of_x_cols, dataset, test_frac=0.2):
    x_train, x_test, y_train, y_test = train_test_split(dataset[name_of_x_cols], dataset[name_of_y_col], test_size=test_frac, random_state=557)
        
    model = classifier_fn(x_train, y_train)
    
    y_pred_train, y_pred = model.predict(x_train), model.predict(x_test)
    
    train_summary, test_summary = summarize_classification(y_train, y_pred_train), summarize_classification(y_test, y_pred)
    
    model_crosstab = pd.crosstab(pd.DataFrame({'y_test': y_test, 'y_pred': y_pred}).y_pred, y_test)
    
    return {'test': test_summary, 'confusion_matrix': model_crosstab}

In [9]:
def compare_results():
    for key in results:
        print('Classification: ', key)
        
        # print()
        # print('Training data')
        # for score in results[key]['training']:
        #     print(score, results[key]['training'][score])
            
        print()
        print('Test data')
        for score in results[key]['test']:
            print(score, results[key]['test'][score])
        
        print()

In [10]:
def logistic_fn(x_train, y_train):
    model = LogisticRegression(penalty='l2', C=1.0, solver='liblinear')
    model.fit(x_train, y_train)
    
    return model

In [11]:
def linear_discriminant_fn(x_train, y_train, solver='svd'):
    model = LinearDiscriminantAnalysis(solver=solver)
    model.fit(x_train, y_train)
    
    return model

In [12]:
def linear_svc_fn(x_train, y_train, C=1.0, max_iter=10000, tol=1e-3):
    model = LinearSVC(C=C, max_iter=max_iter, tol=tol, dual=False)
    model.fit(x_train, y_train)
    
    return model

In [13]:
def radius_neighbors_fn(x_train, y_train, radius=40.0):
    model = RadiusNeighborsClassifier(radius=radius)
    model.fit(x_train, y_train)
    
    return model

In [14]:
def decision_tree_fn(x_train, y_train, max_depth=5, max_features=None):
    model = DecisionTreeClassifier(max_depth=max_depth, max_features=max_features, min_samples_leaf=4, min_samples_split=2, criterion='entropy')
    model.fit(x_train, y_train)
    
    return model

In [15]:
results['logistic'] = build_model(logistic_fn, 'pr_points_bucket', features.columns, df)
results['lda'] = build_model(linear_discriminant_fn, 'pr_points_bucket', features.columns, df)
results['svm'] = build_model(linear_svc_fn, 'pr_points_bucket', features.columns, df)
results['radius_neighbors'] = build_model(radius_neighbors_fn, 'pr_points_bucket', features.columns, df)
results['decision_tree'] = build_model(decision_tree_fn, 'pr_points_bucket', features.columns, df)

compare_results()

Classification:  logistic

Test data
accuracy 0.7444168734491315
precision 0.6659866560820672
recall 0.7444168734491315
f1 0.6871074050380578
accuracy_count 300.0

Classification:  lda

Test data
accuracy 0.7146401985111662
precision 0.6623592967339037
recall 0.7146401985111662
f1 0.6639221105003279
accuracy_count 288.0

Classification:  svm

Test data
accuracy 0.7344913151364765
precision 0.644316187788621
recall 0.7344913151364765
f1 0.6699584823572743
accuracy_count 296.0

Classification:  radius_neighbors

Test data
accuracy 0.7096774193548387
precision 0.5036420395421436
recall 0.7096774193548387
f1 0.5891661594643944
accuracy_count 286.0

Classification:  decision_tree

Test data
accuracy 0.7369727047146402
precision 0.7025396228360373
recall 0.7369727047146402
f1 0.7178712285146931
accuracy_count 297.0



In [16]:
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=557)

param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
grid_search = GridSearchCV(DecisionTreeClassifier(), param_grid, cv=3, return_train_score=True)
grid_search.fit(x_train, y_train)

grid_search.best_params_

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


{'criterion': 'gini',
 'max_depth': 5,
 'min_samples_leaf': 2,
 'min_samples_split': 10}